![Banner](img/AI_Special_Program_Banner.jpg)

## Hyperparameter Optimization (HPO) w/ Optuna - Material
---

***Data Source**: [Kaggle](https://www.kaggle.com/datasets/andrewmvd/fetal-health-classification)*

***Optuna Documentation**: [https://optuna.readthedocs.io/en/stable/](https://optuna.readthedocs.io/en/stable/)*

Hyperparameter optimization (HPO) is a critical aspect of machine learning that involves selecting a set of optimal hyperparameters for a learning algorithm. Hyperparameters are the configuration settings used to tune how the algorithm learns. Unlike model parameters, which are learned during training, hyperparameters are set prior to the training process and have a significant impact on the outcome of machine learning models.

The process of hyperparameter optimization seeks to find the combination of hyperparameters that yields the best performance, as measured by a predefined score or objective function. This is particularly important in complex models like deep neural networks, where the number of hyperparameters can be quite large, and the right set of values can dramatically improve this performance.

Several techniques have been developed for hyperparameter optimization, including:

1. **Grid Search:** A brute-force approach that evaluates all possible combinations of hyperparameters.
2. **Random Search:** Evaluates a random selection of hyperparameters, which can sometimes lead to better results in less time than grid search.
3. **Bayesian Optimization:** Uses a probabilistic model to guide the search for the best hyperparameters and is often more efficient than random or grid search.
4. **Gradient-based Optimization:** Adjusts hyperparameters using gradient descent methods.

The choice of technique depends on the specific problem, the type of model, the nature of the search space, and computational resources. As models and datasets become more complex, efficient hyperparameter optimization becomes increasingly important to achieve the best results.

In this module, we will explore how hyperparameter optimization can be applied to improve overall model performance. We will look at various strategies to do so, understand their trade-offs, and learn how to implement them effectively. Specifically, for this class we will be working with a Python library called Optuna, an open-source hyperparameter optimization framework that automates the process of finding the best hyperparameters.

The contents of this notebook are split into two major parts:
- **Part I**: Solving a classification problem without any automatic hyperparameter tuning (manual tuning possible)
- **Part II**: Solving the same problem, but applying automatic hyperparameter tuning instead

## Overview
---

* [Part I](#Part-I)
* [Part II](#Part-II)
    * [Study & Objective Function](#Study-&-Objective-Function)
    * [Neural Architecture Search (NAS)](#Neural-Architecture-Search-(NAS))
    * [Trial Pruning](#Trial-Pruning)
    * [Optuna Visualizations](#Optuna-Visualizations)
* [Learning Outcomes](#Learning-Outcomes)

## Part I
---

> **Library imports and general settings**:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import CosineAnnealingLR

import torch
import torch.nn as nn

In [ ]:
EPOCHS = 50
CLASSES = 3

torch.manual_seed(42)
random.seed(42)
RANDOM_SEED = 42

> **Definition of (selected) hyperparameters**:

In [ ]:
NN_ARCHITECTURE = [15, 10] # representing the number of neurons for the first (index 0) and second hidden layer (index 1)
BATCH_SIZE = 100
OPTIMIZER = 'Adam' # SGD vs. Adam vs. RMSprop
LEARNING_RATE = 1
WEIGHT_DECAY = 1
SCHEDULER = True

<img src='img/HPO_NN_Architecture.svg'>

> **Data preparation**:

Our chosen use-case is looking at tabular (very unbalanced) health data for fetal health classification (normal, suspect, pathological). However, we are not so much interested in the use-case itself. Instead, we are trying to figure out a way to train a model with optimized parameters. Apart from being unbalanced, the data at hand is already cleaned and does not have any missing values.

In [ ]:
data = pd.read_csv('data/HPO_Data.csv')
data.head()

In [ ]:
len(data)

In [ ]:
data.isnull().any().any()

In [ ]:
data.fetal_health.value_counts().plot(kind='bar')
plt.show()
data.fetal_health.value_counts()

Due to data being this unbalanced, the observed null accuracy is already quite high.

In [ ]:
null_accuracy = data.fetal_health.value_counts()[1.0]/len(data)
null_accuracy

In [ ]:
encoder = OrdinalEncoder()
oe_columns = ['fetal_health']
encoder.fit(data[oe_columns])
data[oe_columns] = encoder.transform(data[oe_columns])

In [ ]:
data.fetal_health.value_counts()

> **Creating helper classes and methods**:

In [ ]:
class FetalHealthData(torch.utils.data.Dataset):
    def __init__(self, data):
        self.labels = data.fetal_health.tolist()
        self.features = data.drop(columns=['fetal_health'], axis=1).values.tolist()
    
    def __getitem__(self, index):
        sample = np.array(self.features[index]), np.array(self.labels[index])
        return sample
        
    def __len__(self):
        return len(self.labels)

In [ ]:
# HP: Hidden Layer 0, Hidden Layer 1
def get_model():
    
    ###
    layer_0 = NN_ARCHITECTURE[0]
    layer_1 = NN_ARCHITECTURE[1]
    ###
    
    layers = list()
    
    # 21 Input Features
    in_features = len(data.drop(columns=['fetal_health'], axis=1).columns)
    
    # Input Layer -> Hidden Layer 0
    layers.append(nn.Linear(in_features, layer_0))
    layers.append(nn.LeakyReLU())
    
    # Hidden Layer 0 -> Hidden Layer 1
    layers.append(nn.Linear(layer_0, layer_1))
    layers.append(nn.LeakyReLU())
    
    # Hidden Layer 1 -> Output Layer (3 Classes)
    layers.append(nn.Dropout())
    layers.append(nn.Linear(layer_1, CLASSES))

    return nn.Sequential(*layers)

In [ ]:
# HP: Batch Size
def get_data():
    
    ###
    batch_size = BATCH_SIZE
    ###
    
    training_data, testing_data = train_test_split(data, test_size=0.2, random_state=RANDOM_SEED, stratify=data.fetal_health)
    training_data, testing_data = FetalHealthData(training_data), FetalHealthData(testing_data)
    return torch.utils.data.DataLoader(training_data, batch_size=batch_size, shuffle=True), torch.utils.data.DataLoader(testing_data, batch_size=batch_size, shuffle=False)

In [ ]:
# HP: Optimizer, Learning Rate, Weight Decay
def get_optimizer(model):
    
    ###
    optimizer = OPTIMIZER
    learning_rate = LEARNING_RATE
    weight_decay = WEIGHT_DECAY
    ###
    
    if optimizer == 'Adam':
        return torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    elif optimizer == 'SGD':
        return torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    elif optimizer == 'RMSprop':
        return torch.optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

> **Creating training loop**:

In [ ]:
# HP: Scheduler
def train(model, training_batches, testing_batches):
    ###
    scheduler = SCHEDULER
    ###
    
    accuracy = list()
    criterion = nn.CrossEntropyLoss()
    optimizer = get_optimizer(model)
    
    if scheduler:
        scheduler = CosineAnnealingLR(optimizer, EPOCHS-1, verbose=False)
    
    for epoch in range(EPOCHS):
        ### Training
        model.train()
        for samples, labels in training_batches:
            optimizer.zero_grad()
            outputs = model(samples.float())
            loss = criterion(outputs, labels.long())
            loss.backward()
            optimizer.step()
            
        if scheduler:
            scheduler.step()
            
        num_samples = 0
        correct_predictions = 0
        ### Testing
        model.eval()
        with torch.no_grad():
            for samples, labels in testing_batches:
                output = model(samples.float())
                correct_predictions += (output.argmax(dim=1) == labels).sum().item()
                num_samples += labels.size(0)
            
        accuracy.append(100.0 * correct_predictions / num_samples)
    
    return accuracy

> **Training & evaluation**:

In [ ]:
model = get_model()
training_batches, testing_batches = get_data()
history = train(model, training_batches, testing_batches)

In [ ]:
plt.plot(history)
plt.ylabel('validation accuracy')
plt.xlabel('epoch')
plt.grid()

In [ ]:
history[-1]

If you made no changes to the preset parameters you can see that the performance of the training outcome is not great. In fact, prediction accuracy does not exceed null accuracy which makes the model rather useless.

**Exercise:** Go back to the initial definition of the hyperparameters at the beginning of this notebook. Finetune those parameters manually. How do you decide for meaningful values? How does your model perform after you manually set new parameters?

## Part II
---

### Study & Objective Function

To get an Optuna pipeline running, usually the first steps should be:
* To define a **study object** which describes the intent and used methodology for the optimization process
* To create an **objective function** that receives updates on the model's fitness/performance during training
* To declare valid ranges and values for selected parameters
* To run the study for a given number of **trials** in which optimization is performed

> **Adding needed libraries**:

In [ ]:
import optuna
import torch.optim as optim

> **Creating a study object (in memory)**:

In [ ]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=RANDOM_SEED))

> **Changing existing function definitions to accept tuned parameters**:

In [ ]:
def get_model(params):
    ###
    layer_0 = params['layer_0']
    layer_1 = params['layer_1']
    ###
    
    layers = list()
    
    # 21 Input Features
    in_features = len(data.drop(columns=['fetal_health'], axis=1).columns)
    
    # Input Layer -> Hidden Layer 0
    layers.append(nn.Linear(in_features, layer_0))
    layers.append(nn.LeakyReLU())
    
    # Hidden Layer 0 -> Hidden Layer 1
    layers.append(nn.Linear(layer_0, layer_1))
    layers.append(nn.LeakyReLU())
    
    # Hidden Layer 1 -> Output Layer (3 Classes)
    layers.append(nn.Dropout())
    layers.append(nn.Linear(layer_1, CLASSES))

    return nn.Sequential(*layers)

In [ ]:
def get_data(params):
    ###
    batch_size = params['batch_size']
    ###
    
    training_data, testing_data = train_test_split(data, test_size=0.2, random_state=RANDOM_SEED, stratify=data.fetal_health)
    training_data, testing_data = FetalHealthData(training_data), FetalHealthData(testing_data)
    return torch.utils.data.DataLoader(training_data, batch_size=batch_size, shuffle=True), torch.utils.data.DataLoader(testing_data, batch_size=batch_size, shuffle=False)

In [ ]:
def get_optimizer(model, params):
    ###
    optimizer = params['optimizer']
    learning_rate = params['learning_rate']
    weight_decay = params['weight_decay']
    ###
    
    if optimizer == 'Adam':
        return torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    elif optimizer == 'SGD':
        return torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    elif optimizer == 'RMSprop':
        return torch.optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    
    # To make things shorter, this would be an alternative way to achieve the same thing:
    # return getattr(optim, optimizer)(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

In [ ]:
def train(model, training_batches, testing_batches, params):
    ###
    scheduler = params['scheduler']
    ###
    
    accuracy = list()
    criterion = nn.CrossEntropyLoss()
    
    ###
    optimizer = get_optimizer(model, params)
    ###
    
    if scheduler:
        scheduler = CosineAnnealingLR(optimizer, EPOCHS-1, verbose=False)
    
    for epoch in range(EPOCHS):
        ### Training
        model.train()
        for samples, labels in training_batches:
            optimizer.zero_grad()
            outputs = model(samples.float())
            loss = criterion(outputs, labels.long())
            loss.backward()
            optimizer.step()
        
        if scheduler:
            scheduler.step()
        
        num_samples = 0
        correct_predictions = 0
        ### Testing
        model.eval()
        with torch.no_grad():
            for samples, labels in testing_batches:
                output = model(samples.float())
                correct_predictions += (output.argmax(dim=1) == labels).sum().item()
                num_samples += labels.size(0)
            
        accuracy.append(100.0 * correct_predictions / num_samples)
    
    return accuracy

> **Defining an objective function that includes hyperparameters and their valid ranges**:

In [ ]:
def objective(trial):
    params = {
        'layer_0': trial.suggest_int('layer_0', 8, 256),
        'layer_1': trial.suggest_int('layer_1', 8, 256),
        'batch_size': trial.suggest_int('batch_size', 8, 128),
        'optimizer': trial.suggest_categorical('optimizer', ['SGD', 'Adam', 'RMSprop']),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
        'weight_decay': trial.suggest_loguniform('weight_decay', 1e-5, 1e-1),
        'scheduler': True if trial.suggest_int('scheduler', 0, 1) == 1 else False
    }
    
    model = get_model(params)
    training_batches, testing_batches = get_data(params)
    history = train(model, training_batches, testing_batches, params)
    
    # Fitness-Value
    return history[-1]

> **Optimizing those parameters**:

In [ ]:
study.optimize(objective, n_trials=5)

In [ ]:
study.best_params

### Neural Architecture Search (NAS)

Optuna features a very powerful **define-by-run** principle which does not limit the user to a static model definition. Instead, it is possible to freely define the search space while the optimization is already being performed. This allows for adaptations to the model design (number of layers, neurons, etc.) - which could be considered a special set of hyperparameters. Because this kind of optimization deals with the network architecture itself, the process is specifically called **neural architecture search**. Following, we implement such a NAS on a very basic level.

> **Changing model definition to be determined during runtime (including architecture related parameters)**:

In [ ]:
def get_model(trial):
    # Suggesting Numbers of Hidden Layers
    num_layers = trial.suggest_int('num_layers', 2, 4)
    
    layers = list()
    
    # 21 Input Features
    in_features = len(data.drop(columns=['fetal_health'], axis=1).columns)
    
    # Suggest Numbers of Neurons for each Hidden Layer
    for layer in range(num_layers):
        out_features = trial.suggest_int(f'layer_{layer}', 8, 256)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.LeakyReLU())
        
        in_features = out_features
    
    layers.append(nn.Dropout())
    layers.append(nn.Linear(in_features, CLASSES))

    return nn.Sequential(*layers)

> **Adapting objective function (removing architecture related parameters)**:

In [ ]:
def objective(trial):
    params = {
        'batch_size': trial.suggest_int('batch_size', 8, 128),
        'optimizer': trial.suggest_categorical('optimizer', ['SGD', 'Adam', 'RMSprop']),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
        'weight_decay': trial.suggest_loguniform('weight_decay', 1e-5, 1e-1),
        'scheduler': True if trial.suggest_int('scheduler', 0, 1) == 1 else False
    }
    
    ###
    model = get_model(trial)
    ###
    
    training_batches, testing_batches = get_data(params)
    history = train(model, training_batches, testing_batches, params)
    
    # Fitness-Value
    return history[-1]

> **Running the study with varying number of hidden layers**:

In [ ]:
study.optimize(objective, n_trials=5)

In [ ]:
study.best_params

> **Continuing parameter search**:

In [ ]:
study.optimize(objective, n_trials=5)

In [ ]:
study.best_params

Remarkably, after only a few trials, we were able to come up with a set of hyperparameters resulting in a majorly improved prediction accuracy.

### Trial Pruning

Another aspect of Optuna's vast functionality, is its ability to let go of unpromising trials. This technique - called pruning - allows to focus on good sets of parameters only and might speed up the optimization process dramatically.

> **Create a new study object that includes a pruning strategy and can be persisted on disc**:

In [ ]:
study = optuna.create_study(
    direction='maximize',
    sampler=optuna.samplers.TPESampler(seed=RANDOM_SEED),
    pruner=optuna.pruners.MedianPruner(),
    study_name='Fetal_Health_Study',
    storage='sqlite:///data/Fetal_Health.sqlite3',
    load_if_exists=True
)

> **Enable feedback on fitness value (accuracy) during training and apply pruning strategy**:

In [ ]:
def train(model, training_batches, testing_batches, params, trial):
    ###
    scheduler = params['scheduler']
    ###
    
    accuracy = list()
    criterion = nn.CrossEntropyLoss()
    
    ###
    optimizer = get_optimizer(model, params)
    ###
    
    if scheduler:
        scheduler = CosineAnnealingLR(optimizer, EPOCHS-1, verbose=False)
    
    for epoch in range(EPOCHS):
        ### Training
        model.train()
        for samples, labels in training_batches:
            optimizer.zero_grad()
            outputs = model(samples.float())
            loss = criterion(outputs, labels.long())
            loss.backward()
            optimizer.step()
            
        if scheduler: 
            scheduler.step()
        
        num_samples = 0
        correct_predictions = 0
        ### Testing
        model.eval()
        with torch.no_grad():
            for samples, labels in testing_batches:
                output = model(samples.float())
                correct_predictions += (output.argmax(dim=1) == labels).sum().item()
                num_samples += labels.size(0)
            
        ###
        accuracy.append(100.0 * correct_predictions / num_samples)
        trial.report(accuracy[-1], epoch)
        
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
        ###
    
    return accuracy

> **Making final adaptations to the objective function**:

In [ ]:
def objective(trial):
    params = {
        'batch_size': trial.suggest_int('batch_size', 8, 128),
        'optimizer': trial.suggest_categorical('optimizer', ['SGD', 'Adam', 'RMSprop']),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
        'weight_decay': trial.suggest_loguniform('weight_decay', 1e-5, 1e-1),
        'scheduler': True if trial.suggest_int('scheduler', 0, 1) == 1 else False
    }
    
    ###
    model = get_model(trial)
    ###
    
    training_batches, testing_batches = get_data(params)
    
    ###
    history = train(model, training_batches, testing_batches, params, trial)
    ###
    
    # Fitness-Value
    return history[-1]

> **Running the newly created study**:

In [ ]:
study.optimize(objective, n_trials=30)

In [ ]:
study.best_params

### Optuna Visualizations

This final section highlights various out-of-the-box visualizations offered by Optuna. Those visualizations can be very helpful when it comes to evaluating the importance of certain parameters in respect to the problem at hand. A comprehensive list of all available visualizations and their respective meaning can be found [here](https://optuna.readthedocs.io/en/stable/reference/visualization/index.html).

In [ ]:
# Matplotlib based visualizations
optuna.visualization.matplotlib.plot_optimization_history(study)

In [ ]:
# Plotly based visualizations
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_intermediate_values(study)

In [ ]:
# Investigating a particular trial
trials = study.get_trials()
trials[14].params

In [ ]:
optuna.visualization.plot_slice(study)

In [ ]:
optuna.visualization.plot_param_importances(study)

## Learning Outcomes
---

By the end of this section you should be able to:

* grasp the fundamental differences between hyperparameters and model parameters, and why hyperparameter tuning is crucial for model performance;
* recognize and list hyperparameters in various machine learning models, including those specific to neural networks & tree-based methods;
* understand various hyperparameter optimization strategies, including Grid Search, Random Search, and Bayesian Optimization;
* utilize Optuna for hyperparameter tuning, understand how to define the search space and run optimization trials in a structured and reproducible manner;
* interpret optimization results, make informed decisions based on those results, and effectively communicate findings and choices.